In [34]:
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms
import torch.nn.functional as F

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
    NameToClassTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [35]:
tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        NameToClassTransform(),
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)

In [36]:
data = dataset[0]  # Get the first graph object.
data.y

tensor(4)

In [37]:
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
# print(f"Number of classes: {dataset.num_classes}")


print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)

Percentage: 0.27, 0.73

Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 1

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 1], y=4)
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


In [38]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")

Number of training graphs: 562
Number of test graphs: 150


In [39]:
train_loader = DataLoader(train_dataset, batch_size=10)
test_loader = DataLoader(test_dataset, batch_size=10)

In [40]:
for batch in train_loader:
    break
batch.y

tensor([0, 1, 2, 1, 1, 4, 4, 0, 3, 3])

In [41]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 5)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x


for batch in train_loader:
    break

model = GCN(hidden_channels=64)
print(model(batch.x, batch.edge_index, batch.batch))

tensor([[-0.3993, -0.0838, -0.4797,  0.1843,  0.1530],
        [ 0.2563, -0.4330, -1.0375,  0.6955, -0.1932],
        [-0.4545, -0.3014,  0.0632,  0.2620, -0.0970],
        [-0.5419,  0.1279, -0.4979,  0.2693,  0.0587],
        [ 0.3741, -0.0875, -0.0427,  0.6432,  0.7742],
        [-0.0616, -0.1327, -0.1828,  0.2206,  0.0559],
        [ 0.4510,  0.2749,  0.0296,  0.2863,  0.4205],
        [ 0.2428,  0.1032, -0.1836,  0.9172,  0.7433],
        [ 0.2160,  0.6167, -0.4848,  0.4869,  0.3303],
        [ 0.0614,  0.1672,  0.0395,  0.5327,  0.7759]],
       grad_fn=<AddmmBackward0>)


In [42]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(
            data.x, data.edge_index, data.batch
        )  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int(
            (pred == data.y).sum()
        )  # Check against ground-truth labels.
    return correct / len(
        loader.dataset
    )  # Derive ratio of correct predictions.


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 0.2758, Test Acc: 0.2533
Epoch: 002, Train Acc: 0.5125, Test Acc: 0.5133
Epoch: 003, Train Acc: 0.8238, Test Acc: 0.8467
Epoch: 004, Train Acc: 0.8256, Test Acc: 0.8467
Epoch: 005, Train Acc: 0.7491, Test Acc: 0.7867
Epoch: 006, Train Acc: 0.8274, Test Acc: 0.8467
Epoch: 007, Train Acc: 0.8256, Test Acc: 0.8467
Epoch: 008, Train Acc: 0.8256, Test Acc: 0.8467
Epoch: 009, Train Acc: 0.8256, Test Acc: 0.8467


KeyboardInterrupt: 